## 1. 필요한 모듈 설치

In [1]:
!sudo apt install cmake
!pip3 install face_recognition

Reading package lists... Done
Building dependency tree       
Reading state information... Done
cmake is already the newest version (3.10.2-1ubuntu2.18.04.2).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [2]:
import os
import matplotlib.pyplot as plt
import matplotlib.image as img
import face_recognition
import numpy as np 
import pandas as pd
import seaborn as sns
from PIL import Image 

RuntimeError: ignored

## 2. 이미지 준비

In [ ]:
dir_path = '/content/drive/MyDrive/Github/AIFFEL/E05/'
my_dir_path = '/content/drive/MyDrive/Github/AIFFEL/E05/me'
cel_dir_path = '/content/drive/MyDrive/Github/AIFFEL/E05/celebrity'
file_list = os.listdir(my_dir_path)

print ("file_list: {}".format(file_list))

#Set figsize here
fig, axes = plt.subplots(nrows=3, ncols=1, figsize=(12,10))

# flatten axes for easy iterating
for i, ax in enumerate(axes.flatten()):
  image = img.imread(my_dir_path+'/'+file_list[i])
  ax.imshow(image)
plt.show()

fig.tight_layout()

## 3. 얼굴 영역 자르기

In [ ]:
def get_cropped_face(image_file):
    image = face_recognition.load_image_file(image_file)
    face_locations = face_recognition.face_locations(image)

    a, b, c, d = face_locations[0]
    cropped_face = image[a:c,d:b,:]
    
    return cropped_face

In [ ]:
# test 코드
image_path = '/content/drive/MyDrive/Github/AIFFEL/E05/me/jungin2.jpg'

cropped_face = get_cropped_face(image_path)
plt.imshow(cropped_face)

## 4. 얼굴 영역 임베딩 추출

In [ ]:
# 얼굴 영역을 가지고 얼굴 임베딩 벡터를 구하는 함수
def get_face_embedding(face):
    return face_recognition.face_encodings(face)

In [ ]:
def get_face_embedding_dict(dir_path):
    file_list = os.listdir(dir_path)
    embedding_dict = {}
    
    for file in file_list:
        img_path = f'{dir_path}/{file}'
        try:
          face = get_cropped_face(img_path)
          embedding = get_face_embedding(face)
          if len(embedding) > 0:  
          # 얼굴영역 face가 제대로 detect되지 않으면  len(embedding)==0인 경우가 발생하므로 
          # os.path.splitext(file)[0]에는 이미지파일명에서 확장자를 제거한 이름이 담깁니다. 
              embedding_dict[os.path.splitext(file)[0]] = embedding[0]
        except:
            pass
    return embedding_dict

In [ ]:
cel_embedding_dict = get_face_embedding_dict(cel_dir_path)  # celebrity face embedding dictionary
my_embedding_dict = get_face_embedding_dict(my_dir_path)    # my face embedding dictionary
cel_embedding_dict.update(my_embedding_dict) #병합
embedding_dict = cel_embedding_dict

## 5. 임베딩 벡터 간 거리 구하기

In [ ]:
#두 이미지(사람 이름) 사이의 임베딩 벡터 거리를 구하는 함수
def get_distance(name1, name2):
    return np.linalg.norm(embedding_dict[name1]-embedding_dict[name2], ord=2)

In [ ]:
#name1과 name2의 거리를 비교, 반환
def get_sort_key_func(name1):
    def get_distance_from_name1(name2):
        return get_distance(name1, name2)
    return get_distance_from_name1

In [ ]:
#임의의 이름에 대해 다른 이미지의 임베딩 거리를 정렬해서 출력하는 함수
def get_nearest_face(name, total_image=len(embedding_dict.keys())):
    sort_key_func = get_sort_key_func(name)
    sorted_faces = sorted(embedding_dict.items(), key=lambda x:sort_key_func(x[0]))
    
    i = 0
    rank = 1
    #while i < total_image:
    while rank<=10:
      if 'jungin' in sorted_faces[i][0]: #except for my name
        i+=1
        continue
      if sorted_faces[i]:
          print('순위 {} : 이름: {}, 거리({})'.format(rank, sorted_faces[i][0], sort_key_func(sorted_faces[i][0])))
          i+=1
          rank+=1

In [ ]:
get_nearest_face('jungin')

In [ ]:
get_nearest_face('jungin1')

In [ ]:
get_nearest_face('jungin2')

## 6. 시각화